In [2]:
import json
from collections import Counter, defaultdict, deque
import gc

import numpy as np
import pandas as pd
import glob
import time
from tqdm import tqdm

from sklearn import model_selection, metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sn

import lightgbm as lgb
from scipy.stats import skew

import feather
import pickle
import h5py

from concurrent import futures
#import riiideducation

In [3]:
pocket_1221 = pd.read_csv("./pred_1221.csv")
pocket_1223 = pd.read_csv("./pred_1223.csv")
pocket_1226 = pd.read_csv("./pred_1226.csv")
owruby = pd.read_csv("./owruby_v022.csv")
sakami = pd.read_csv("./sakami-1221.csv")
owruby2_1 = pd.read_csv("./owruby_v026_1.csv")
owruby2_2 = pd.read_csv("./owruby_v026_2.csv")
owruby2_3 = pd.read_csv("./owruby_v026_3.csv")
owruby2_4 = pd.read_csv("./owruby_v026_4.csv")

In [15]:
sakami2 = np.load("sakami-134pp.npy")
sakami["sakami2"] = sakami2

In [18]:
sakami.head()

,Unnamed: 0,row_id,answered_correctly,prediction,sakami2
0,96,96,1,0.297885,0.297187
1,97,97,0,0.318944,0.304867
2,98,98,1,0.551568,0.509313
3,99,99,0,0.494178,0.516816
4,100,100,1,0.144016,0.120744


In [4]:
#sakami["row_id"].describe()

In [5]:
print(pocket_1226.head(2))
print(owruby2_1.head(2))
print(sakami.head(2))

          0
0  0.871062
1  0.866829
     row_id  answered_correctly  label
0  70629059            0.818395      1
1  70629060            0.817051      1
   Unnamed: 0  row_id  answered_correctly  prediction
0          96      96                   1    0.297885
1          97      97                   0    0.318944


In [16]:
merged = owruby.copy()
merged["pocket_1221"] = pocket_1221
merged["pocket_1223"] = pocket_1223
merged["pocket_1226"] = pocket_1226
merged["owruby"] = merged["answered_correctly"]
merged["owruby2_1"] = owruby2_1["answered_correctly"]
merged["owruby2_2"] = owruby2_2["answered_correctly"]
merged["owruby2_3"] = owruby2_3["answered_correctly"]
merged["owruby2_4"] = owruby2_4["answered_correctly"]
merged.drop(columns="answered_correctly", inplace=True)

In [17]:
merged2 = pd.merge(merged, sakami, on="row_id", how="inner")

In [19]:
print(merged.shape, merged2.shape)

(5000000, 10) (5000000, 14)


In [20]:
merged2["sakami"] = merged2["prediction"]
merged2.drop(columns="prediction", inplace=True)

In [21]:
merged2["owruby2"] = (merged2["owruby2_1"] + merged2["owruby2_2"] + merged2["owruby2_3"] + merged2["owruby2_4"]) / 4

In [22]:
merged2.head(2)

,row_id,label,pocket_1221,pocket_1223,pocket_1226,owruby,owruby2_1,owruby2_2,owruby2_3,owruby2_4,Unnamed: 0,answered_correctly,sakami2,sakami,owruby2
0,70629059,1,0.899222,0.882067,0.871062,0.851218,0.818395,0.762108,0.795984,0.824208,70629059,1,0.894440,0.900553,0.800174
1,70629060,1,0.875296,0.861982,0.866829,0.832965,0.817051,0.821449,0.836079,0.826704,70629060,1,0.860161,0.865924,0.825321


In [13]:
# pred_col = [
#     "pocket_1221", "pocket_1223", "owruby", "sakami"
# ]

In [23]:
merged2["avg2"] = (merged2["sakami2"] + merged2["pocket_1226"]) / 2
merged2["avg3"] = (merged2["sakami2"] + merged2["pocket_1223"]) / 2

In [24]:
metrics.roc_auc_score(merged2["label"], merged2["avg2"])

0.8108916699904363

In [25]:
metrics.roc_auc_score(merged2["label"], merged2["avg3"])

0.8108136239937651

In [37]:
merged2["avg"] = (merged2["sakami2"] + merged2["owruby2"]) / 2
merged2["3avg"] = (merged2["sakami2"] + merged2["owruby2"] + merged2["pocket_1226"]) / 3
merged2["weighted"] = merged2["sakami2"]*0.4 + merged2["owruby2"]*0.4 + merged2["pocket_1226"]*0.2

In [27]:
metrics.roc_auc_score(merged2["label"], merged2["sakami"])

0.8085990634123565

In [38]:
metrics.roc_auc_score(merged2["label"], merged2["avg"])

0.8116706471505933

In [29]:
metrics.roc_auc_score(merged2["label"], merged2["3avg"])

0.812223227247307

In [30]:
metrics.roc_auc_score(merged2["label"], merged2["weighted"])

0.8123557143167384

In [31]:
merged2["sakami-rank"] = merged2["sakami2"].rank()
merged2["owruby-rank"] = merged2["owruby2"].rank()
merged2["pocket-rank"] = merged2["pocket_1226"].rank()

In [32]:
merged2["ranked_avg"] = (merged2["sakami-rank"] + merged2["owruby-rank"])
merged2["ranked_3avg"] = (merged2["sakami-rank"] + merged2["owruby-rank"] + merged2["pocket-rank"])

In [33]:
metrics.roc_auc_score(merged2["label"], merged2["ranked_avg"])

0.8117137458640472

In [34]:
metrics.roc_auc_score(merged2["label"], merged2["ranked_3avg"])

0.8122063305623283

In [39]:
for power in [1.5, 1.75, 2, 2.25, 2.5, 3, 3.5, 4]:
    merged2["power_mean"] = (merged2["sakami2"]**power + merged2["owruby2"]**power + merged2["pocket_1226"]**power)
    score = metrics.roc_auc_score(merged2["label"], merged2["power_mean"])
    print(power, score)

1.5 0.8122420446445264
1.75 0.8122429171510669
2 0.8122395856489596
2.25 0.8122324370676334
2.5 0.8122222494209007
3 0.8121941845373124
3.5 0.8121586996153545
4 0.8121174458482582
